In [1]:
import sklearn
print(sklearn.__version__)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/utils/_param_validation.py:11: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 1.22.4)
  from scipy.sparse import csr_matrix, issparse


1.5.1


In [17]:
import pandas as pd
import sagemaker
from sagemaker.session import Session
from sagemaker.sklearn import SKLearn
import boto3

In [5]:
!pip install s3fs

In [6]:
df = pd.read_csv("s3://projects-abdimatin/RandomClassifier/data/HR_comma_sep.csv")

In [7]:
df.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,Department,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [8]:
df.drop(["last_evaluation", "number_project", "time_spend_company", "Work_accident", "Department"], axis=1, inplace=True)

In [9]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df["Salary_E"] = le.fit_transform(df["salary"])

In [10]:
df.head()

,satisfaction_level,average_montly_hours,left,promotion_last_5years,salary,Salary_E
0,0.38,157,1,0,low,1
1,0.80,262,1,0,medium,2
2,0.11,272,1,0,medium,2
3,0.72,223,1,0,low,1
4,0.37,159,1,0,low,1


In [11]:
df.drop(["salary"], axis=1, inplace=True)

In [12]:
df.head()

,satisfaction_level,average_montly_hours,left,promotion_last_5years,Salary_E
0,0.38,157,1,0,1
1,0.80,262,1,0,2
2,0.11,272,1,0,2
3,0.72,223,1,0,1
4,0.37,159,1,0,1


In [18]:
cols = ["left"] + [col for col in df.columns if col != "left"]
print(cols)

['left', 'satisfaction_level', 'average_montly_hours', 'promotion_last_5years', 'Salary_E']


In [14]:
sagemaker_client = boto3.client("sagemaker")

In [15]:
df.shape

(14999, 5)

In [19]:
X = df.drop("left", axis=1)
y = df.left

In [16]:
df.head()

,satisfaction_level,average_montly_hours,left,promotion_last_5years,Salary_E
0,0.38,157,1,0,1
1,0.80,262,1,0,2
2,0.11,272,1,0,2
3,0.72,223,1,0,1
4,0.37,159,1,0,1


In [20]:
df = df[cols]

In [21]:
df.head()

,left,satisfaction_level,average_montly_hours,promotion_last_5years,Salary_E
0,1,0.38,157,0,1
1,1,0.80,262,0,2
2,1,0.11,272,0,2
3,1,0.72,223,0,1
4,1,0.37,159,0,1


In [27]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, stratify=df.left, test_size=0.2, random_state=42)

In [19]:
train_df.to_csv("s3://projects-abdimatin/RandomClassifier/data/train-hr.csv", index=False)
test_df.to_csv("s3://projects-abdimatin/RandomClassifier/data/test-hr.csv", index=False)

In [29]:
features = list(train_df.columns)
label = features[0]
print(label)

left


In [30]:
%%writefile script.py
import os
import boto3
import joblib
import argparse
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

def model_fn(model_dir):
    clf = joblib.load(os.path.join(model_dir, "model.joblib"))
    return clf

if __name__ == "__main__":
    print("[INFO] Extracting arguments")
    parser = argparse.ArgumentParser()

    parser.add_argument("--model-dir", type=str, default=os.environ.get("SM_MODEL_DIR"))
    parser.add_argument("--train", type=str, default=os.environ.get("SM_CHANNEL_TRAIN"))
    parser.add_argument("--test", type=str, default=os.environ.get("SM_CHANNEL_TEST"))
    parser.add_argument("--train-file", type=str, default="train-hr.csv")
    parser.add_argument("--test-file", type=str, default="test-hr.csv")

    args, _ = parser.parse_known_args()

    print("[INFO] Reading data")

    # Ensure the correct data source for test data
    train_df = pd.read_csv(os.path.join(args.train, args.train_file))
    test_df = pd.read_csv(os.path.join(args.test, args.test_file))

    features = list(train_df.columns)
    label = features[0]
    print("Building Training and testing datasets")
    
    X_train = train_df.drop(label, axis=1)
    y_train = train_df[label]

    X_test = test_df.drop(label, axis=1)
    y_test = test_df[label]

    model = RandomForestClassifier(n_estimators=100)  # Increased n_estimators for better performance
    model.fit(X_train, y_train)
    
    model_path = os.path.join(args.model_dir, "model.joblib")
    joblib.dump(model, model_path)
    print("Model persisted at " + model_path)

    y_pred = model.predict(X_test)
    print(classification_report(y_test, y_pred))


Writing script.py


In [31]:
role = sagemaker.get_execution_role()
print(role)

arn:aws:iam::557690604891:role/service-role/AmazonSageMaker-ExecutionRole-20241019T114685


In [32]:
from sagemaker.sklearn import SKLearn
FRAMEWORKVERSION = "0.23-1"
estimator = SKLearn(
    entry_point="script.py",
    role = role,
    framework_version=FRAMEWORKVERSION,
    instance_type = "ml.m5.large",
    instance_count=1,
    base_job_name = "RF-custom-sklearn",
    output_path = "s3://projects-abdimatin/RandomClassifier/models",
    use_spot_instance=True,
    wait=True
    )

In [33]:
test_path = "s3://projects-abdimatin/RandomClassifier/data"
train_path = "s3://projects-abdimatin/RandomClassifier/data"

In [34]:
estimator.fit({"train": train_path, "test": test_path})

INFO:sagemaker:Creating training-job with name: RF-custom-sklearn-2024-10-22-10-42-49-149


2024-10-22 10:42:50 Starting - Starting the training job...
2024-10-22 10:43:04 Starting - Preparing the instances for training...
2024-10-22 10:43:30 Downloading - Downloading input data...
2024-10-22 10:44:00 Downloading - Downloading the training image...
2024-10-22 10:44:46 Training - Training image download completed. Training in progress.
2024-10-22 10:44:46 Uploading - Uploading generated training model2024-10-22 10:44:41,138 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2024-10-22 10:44:41,142 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-10-22 10:44:41,186 sagemaker_sklearn_container.training INFO     Invoking user training script.
2024-10-22 10:44:41,362 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-10-22 10:44:41,374 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-10-22 10:44:41,386 sagemaker-training-toolkit INFO

In [37]:
sm_boto = boto3.client("sagemaker")
estimator.latest_training_job.wait(logs="None")
artifact = sm_boto.describe_training_job(
    TrainingJobName=estimator.latest_training_job.name
)['ModelArtifacts']['S3ModelArtifacts']
print(artifact)


2024-10-22 10:44:59 Starting - Preparing the instances for training
2024-10-22 10:44:59 Downloading - Downloading the training image
2024-10-22 10:44:59 Training - Training image download completed. Training in progress.
2024-10-22 10:44:59 Uploading - Uploading generated training model
2024-10-22 10:44:59 Completed - Training job completed
s3://projects-abdimatin/RandomClassifier/models/RF-custom-sklearn-2024-10-22-10-42-49-149/output/model.tar.gz


In [41]:
from sagemaker.sklearn.model import SKLearnModel
from time import gmtime, strftime

model_name = "Custom-Random-Classifier-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model = SKLearnModel(
    name=model_name,
    model_data=artifact,
    role=role,
    entry_point="script.py",
    framework_version=FRAMEWORKVERSION
)
model

In [42]:
endpoint_name = "Custom-Random-Classifier-endpoint-1"
print("Endpoint Nanme: ", endpoint_name)

Endpoint Nanme:  Custom-Random-Classifier-endpoint-1


In [45]:
predictor = model.deploy(
    endpoint_name=endpoint_name, 
    instance_type="ml.m4.xlarge",
    initial_instance_count=1
)

INFO:sagemaker:Creating model with name: Custom-Random-Classifier-2024-10-22-11-01-23
INFO:sagemaker:Creating endpoint-config with name Custom-Random-Classifier-endpoint-1
INFO:sagemaker:Creating endpoint with name Custom-Random-Classifier-endpoint-1


------!

In [60]:
testx = train_df[features[1:]]  # All columns except the label
testy = train_df[features[0]]

In [61]:
from sklearn.metrics import classification_report
pred = predictor.predict(testx)
print(classification_report(y_pred=pred, y_true=testy))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99      9142
           1       0.95      0.98      0.96      2857

    accuracy                           0.98     11999
   macro avg       0.97      0.98      0.98     11999
weighted avg       0.98      0.98      0.98     11999

